In [3]:
!pip install wget
!pip install scikit-learn==1.5

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=7f17bf64362e0f796e15de7cef4e7b9cb76923af50f0f6e38978b68ebcdf43d1
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 65.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.2
    Uninstalling scikit-learn-1.5.2:
      Successfully uninstalled scikit-learn-1.5.2


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import wget
import time as time
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mutual_info_score
from sklearn.metrics import root_mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb

#your import here

## Import Dataset

In [2]:
wget.download('https://github.com/MIE451-2024/course-datasets/raw/refs/heads/main/sale_data.zip', 'sale_data.zip')

'sale_data (1).zip'

In [3]:
!unzip sale_data.zip
DATA_DIR = 'sale_data'

Archive:  sale_data.zip
replace __MACOSX/._sale_data? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [4]:
# get the train dataset (this data is the training data used to train your model, traing data will be same as in our evaluation)
train_data = pd.read_csv(DATA_DIR+"/train_data_assignment.csv")
# get a sample test dataset (the final test data will be different in our evaluation)
test_data = pd.read_csv(DATA_DIR+"/test_data_assignment.csv")
# weather data (optional)
weather_data = pd.read_csv(DATA_DIR+"/weather_data.csv")

## Q4

In [ ]:
# You can use theis code block to test different models and hyperparameters using train_data
# The autograder will only retreived the functions defined below

In [9]:
def data_cleaning(raw_data, weather_data = None):
    # This function should handle the cleaning of the dataset for both training and testing

    # DO NOT modify the name or drop the 'Sales' column when cleaning the training set

    # This function also be able to handle cleaning test dataset, which does not have 'Sales' column
    clean_data = raw_data.copy()
    ################# YOU CODE HERE ################
    datetime_cols = clean_data.select_dtypes(include=['datetime64']).columns
    for col in datetime_cols:
        clean_data[col] = pd.to_datetime(clean_data[col], format='%Y-%m-%d')
        clean_data[f'{col}_Year'] = clean_data[col].dt.year
        clean_data[f'{col}_Month'] = clean_data[col].dt.month
        clean_data[f'{col}_Day'] = clean_data[col].dt.day
        clean_data.drop(columns=[col], inplace=True)

    categorical_cols = clean_data.select_dtypes(include=['object']).columns
    clean_data = pd.get_dummies(clean_data, columns=categorical_cols, drop_first=True)

    if 'Sales' in clean_data.columns:
        clean_data['Sales'] = clean_data['Sales'].fillna(0)  # Handle missing Sales data

    clean_data = clean_data.fillna(0)

    if weather_data is not None and 'State' in clean_data.columns:
        clean_data = clean_data.merge(weather_data, on='State', how='left')

    # the return dataset should be a dataframe
    # that was cleaned and ready to be used for either training or testing
    return clean_data

In [10]:
def train_model(train_data):
    # Split the data into observed features and target feature
    X = train_data.drop(['Sales'], axis=1)
    y = train_data['Sales']


    ################# YOU CODE HERE ################
    # You should:
    # 1. Create your best model and store in the model variable
    # 2. Train your model with the train data where X is the observed features and y is the target feature

    # base_model = linear_model.Ridge()
    # param_grid = {'alpha': [0.1,1,5]}  # Values of alpha from 0.001 to 1000

    # # Use GridSearchCV to find the best alpha using cross-validation
    # scorer = make_scorer(root_mean_squared_error, greater_is_better=False)  # RMSE scorer
    # grid_search = GridSearchCV(estimator=base_model, param_grid=param_grid, scoring=scorer, cv=5)

    # # Fit the model on the training data
    # grid_search.fit(X, y)

    # # Get the best model after tuning alpha
    # model = grid_search.best_estimator_

    '''
    model = linear_model.Ridge(alpha = 5.5)
    rmse = cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_error')
    mean_rmse = np.mean(np.sqrt(-rmse))
    model.fit(X, y)
    '''
    '''
    model = linear_model.Ridge(5.5)
    model.fit(X,y)
    '''

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    model = xgb.XGBRegressor(
        n_estimators=150,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='reg:squarederror',
        random_state=42,
        alpha=5,
        gamma=1,
        reg_lambda=1
        )

    # rmse = cross_val_score(model, X, y, cv=3, scoring='neg_root_mean_squared_error')
    # mean_rmse = np.mean(np.sqrt(-rmse))

    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    y_train_pred = model.predict(X_train)
    train_rmse = np.sqrt(root_mean_squared_error(y_train, y_train_pred))
    # print(f'Training RMSE: {train_rmse:.4f}')

    y_val_pred = model.predict(X_val)
    rmse = np.sqrt(root_mean_squared_error(y_val, y_val_pred))
    # print(f'Validation RMSE: {rmse:.4f}')

    # the model return by this function should be ready to predict the sales given the test data
    return model

In [7]:
def prediction(train_data, test_data):
    # This function should return the prediction given the test dataset
    # The output should be a dataframe with one column
    # and the index should match the test_data index
    train_data = data_cleaning(raw_data = train_data, weather_data = weather_data) #make sure the returned train data as the same order of rows as the train data
    model = train_model(train_data) # This will run the train_model your implemented above to obtain a trained model
    test_data = data_cleaning(raw_data = test_data, weather_data = weather_data) #make sure the returned test data as the same order of rows as the train data

    # check if the test data has all the columns that the model was trained on
    for column in train_data.columns:
        if column not in test_data.columns and column != 'Sales':
            # add the missing column to the test data and set it to 0
            #  (feel free to change it to any other value)
            test_data[column] = 0

    # Ensure the order of the columns in the test dataset is the same as the train
    test_data = test_data[[column for column in train_data.columns if column != 'Sales']]

    ################# YOU CODE HERE ################
    # You should use 'model' to make predictions
    y_pred = model.predict(test_data) # Please store the prediction of test data in y_pred

    # the return should be a dataframe with one column and the index should match test_data index
    return pd.DataFrame(y_pred, columns=['Sales'], index=test_data.index)


In [11]:
# This is the sample code for our autograder to eveluate your model, you should make sure the code is working as expected

# This will be a sample test dataset to help you validate your code, the actual test dataset will be different
X_test = test_data.drop('Sales', axis=1)
# This will be target value that you will compare with your prediction (you will not have this in the test set)
y_test = test_data['Sales']

# THE RUNTIME FOR FOLLOWING CODE SHOULD BE WITHIN 7 MINUTES
y_pred = prediction(train_data, X_test)

# This will print the RMSE score for the test dataset
print(f"RMSE: {root_mean_squared_error(y_test, y_pred)}")

RMSE: 880.897338960675


A clear and concise description of your chosen feature set and any additional features you used through feature engineering or external sources.

Coverted date-based features to datetime format (year, month, day), one hot encoded categorial features, integrated weather data

The name of the regression model you choose:

XGBoost Regressor

Why you chose the feature set and model?

Feature Set:
- Extracted date time features to better encapture patterns

Model:
- Chose since there were three regularization hyperparamters that could be tuned
- Lowest rmse due to model's flexibility for non-linear relationships